<a href="https://colab.research.google.com/github/DavidSchneider47/National-TOD-Atlas/blob/main/OSM_Land_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install the OSM API**

In [1]:
pip install overpy

### **Identify Selected Land Uses 1/4 Mile Around Stations and Calculate the Land Cover**

In [2]:
import csv
import overpy
from shapely.geometry import Polygon
from pyproj import Transformer

def query_area_for_tags(lat, lon, radius_meters, tags):
    api = overpy.Overpass()

    # Build the query for the given tags
    query = "[out:json];("
    for tag in tags:
        query += f'way[{tag}](around:{radius_meters},{lat},{lon});'
    query += ");out body;>;out skel qt;"

    # Execute the query
    result = api.query(query)

    # Calculate total area
    total_area = 0.0
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    for way in result.ways:
        coords = [(node.lon, node.lat) for node in way.nodes]
        if len(coords) < 4:
            continue  # Skip ways with insufficient nodes to form a polygon
        projected_coords = [transformer.transform(lon, lat) for lon, lat in coords]
        polygon = Polygon(projected_coords)
        total_area += polygon.area

    return total_area

vector_tags = {
    "Building Footprint": ["building"],
    "Highway": ["highway=motorway", "highway=motorway_link"],
    "Trunk Road": ["highway=trunk", "highway=trunk_link"],
    "Primary Road": ["highway=primary", "highway=primary_link"],
    "Secondary Road": ["highway=secondary", "highway=secondary_link"],
    "Tertiary Road": ["highway=tertiary", "highway=tertiary_link"],
    "Residential Road": ["highway=residential"],
    "Railway": ["railway=rail"],
    "Subway Railway": ["railway=subway","bridge=yes", "embankment=yes"],
    "Parking Lot": ["amenity=parking"],
    "Residential Area": ["landuse=residential"],
    "Commercial Area": ["landuse=commercial"],
    "Retail Area": ["landuse=retail"],
    "Industrial Area": ["landuse=industrial"],
    "Railway Area": ["landuse=railway"],
    "Cemetery": ["landuse=cemetery"],
    "Woodland": ["natural=wood", "landuse=forest"],
    "Park": ["leisure=park"],
    "Recreation": ["landuse=recreation_ground","leisure=playground", "leisure=pitch","leisure=track"],
    "Education": ["amenity=kindergarten", "amenity=school","amenity=college","amenity=university"],
    "Health Care": ["amenity=hospital","amenity=clinic"],
    "Water": ["natural=water"],
    "Beach": ["natural=beach"],
}

# Load coordinates and Facility ID from the provided CSV
coordinates = []
facility_ids = []
with open('/content/MTA_NYCT2.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        # Extract the 'coordinates' column and split it into lat, lon
        coords_string = row['coordinates']  # Adjust the key if the column name is different
        coords_string = coords_string.strip('()')  # Remove the parentheses
        lat_str, lon_str = coords_string.split(',')
        lat = float(lat_str.strip())
        lon = float(lon_str.strip())
        coordinates.append((lat, lon))
        # Also store the Facility ID
        facility_ids.append(row['Facility ID'])  # Adjust if the column name is different

radius = 402.34  # 1/4 mile in meters

# Check if the output CSV exists to decide on writing headers
file_exists = False
try:
    with open('osm_results_transposed.csv', 'r') as f:
        file_exists = True
except FileNotFoundError:
    pass

with open('osm_results_transposed.csv', 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row only if the file didn't exist
    if not file_exists:
        headers = ["Facility ID", "Latitude", "Longitude"] + list(vector_tags.keys())
        csvwriter.writerow(headers)

    # Loop through each coordinate pair and fetch results
    for facility_id, (lat, lon) in zip(facility_ids, coordinates):
        results_dict = {"Facility ID": facility_id, "Latitude": lat, "Longitude": lon}
        for vector, tags in vector_tags.items():
            area = query_area_for_tags(lat, lon, radius, tags)
            results_dict[vector] = f"{area:.2f}"

        # Write the results row for the current coordinate pair
        row_data = [results_dict[header] for header in headers]
        csvwriter.writerow(row_data)

print("Results appended to osm_results_transposed.csv")


Results appended to osm_results_transposed.csv


### **END**